In [5]:
import numpy as np
import pandas as pd
from IPython.display import display

# Parameters
initial_salary = 70000
hike_rate = 0.035
self_contribution_rate = 0.03
years = 35
start_age = 30
acc_levy = 0.0167
inflation_rate = 0.025

# NZ tax brackets
nz_tax_brackets = [
    (0, 15600), (15601, 53500), (53501, 78100), (78101, 180000), (180001, float('inf'))
]
tax_rates = [0.105, 0.175, 0.30, 0.33, 0.39]

# Fund details
funds = {
    "Harboursafe": {"growth_rate": 0.0375, "volatility": 0.05, "weight": 0.4},
    "Horizon": {"growth_rate": 0.065, "volatility": 0.105, "weight": 0.3},
    "SkyHigh": {"growth_rate": 0.1025, "volatility": 0.2075, "weight": 0.3},
}

# Tax calculation function
def calculate_tax(salary, base_slabs, year, inflation_rate=0.025):
    adjusted_slabs = [
        (bracket[0] * ((1 + inflation_rate) ** (year - 1)),
         bracket[1] * ((1 + inflation_rate) ** (year - 1)), rate)
        for (bracket, rate) in zip(base_slabs, tax_rates)
    ]
    tax = 0
    for lower, upper, rate in adjusted_slabs:
        if salary > lower:
            taxable_income = min(salary, upper) - lower
            tax += taxable_income * rate
        else:
            break
    return tax

# Run simulation
np.random.seed(42)
records = []
salary = initial_salary
corpus = 0

for year in range(1, years + 1):
    age = start_age + year - 1

    # --- Salary growth ---
    if year > 1:
        salary *= (1 + np.random.normal(loc=hike_rate, scale=0.01))

    # --- Deductions ---
    tax = calculate_tax(salary, nz_tax_brackets, year, inflation_rate)
    acc = acc_levy * salary
    net_salary = salary - tax - acc

    # --- Contributions ---
    emp_contribution = self_contribution_rate * net_salary
    employer_contribution = 0.03 * net_salary
    total_contribution = emp_contribution + employer_contribution

    # --- Apply growth to existing corpus ---
    weighted_return = 0
    fund_growth = {}
    for name, fund in funds.items():
        annual_return = np.random.normal(fund["growth_rate"], fund["volatility"])
        fund_growth[f"{name} Return"] = annual_return
        weighted_return += fund["weight"] * annual_return

    corpus = corpus * (1 + weighted_return) + total_contribution

    # --- Calculate fund-wise allocation of current year's contribution ---
    fund_values = {
        name + " Value": total_contribution * (1 + fund_growth[name + " Return"]) * fund["weight"]
        for name, fund in funds.items()
    }

    record = {
        "Year": year,
        "Age": age,
        "Gross Salary": round(salary, 2),
        "Income Tax": round(tax, 2),
        "ACC Levy": round(acc, 2),
        "Net Salary": round(net_salary, 2),
        "Employee Contribution": round(emp_contribution, 2),
        "Employer Contribution": round(employer_contribution, 2),
        "Total Contribution": round(total_contribution, 2),
        "Harboursafe Return": round(fund_values["Harboursafe Value"], 2),
        "Horizon Return": round(fund_values["Horizon Value"], 2),
        "SkyHigh Return": round(fund_values["SkyHigh Value"], 2),
        "Total Fund Value": round(corpus, 2)
    }

    records.append(record)

# Build DataFrame
df_detailed = pd.DataFrame(records)

cumulative_data = df_detailed[[
    "Gross Salary", "Income Tax", "ACC Levy", "Net Salary",
    "Employee Contribution", "Employer Contribution", "Total Contribution",
    "Harboursafe Return", "Horizon Return", "SkyHigh Return"
]].sum()

# Get final fund value only from last row
cumulative_data["Total Fund Value"] = df_detailed["Total Fund Value"].iloc[-1]

cumulative_summary_df = pd.DataFrame([cumulative_data.round(2)])
cumulative_summary_df.insert(0, "Year", "Total")
cumulative_summary_df.insert(1, "Age", "N/A")


# Every 5-year snapshot
summary_every_5_years = df_detailed[df_detailed["Year"] % 5 == 0]

# Display nicely (in Jupyter or IPython)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")
print("🔹 Detailed Yearly Summary:")
display(df_detailed)

print("\n🔹 Cumulative Summary:")
display(cumulative_summary_df)

print("\n🔹 Every 5-Year Summary:")
display(summary_every_5_years)


🔹 Detailed Yearly Summary:


,Year,Age,Gross Salary,Income Tax,ACC Levy,Net Salary,Employee Contribution,Employer Contribution,Total Contribution,Harboursafe Return,Horizon Return,SkyHigh Return,Total Fund Value
0,1,30,"70,000.00","13,220.03","1,169.00","55,610.97","1,668.33","1,668.33","3,336.66","1,417.86","1,051.53","1,238.13","3,336.66"
1,2,31,"73,516.12","14,080.36","1,227.72","58,208.04","1,746.24","1,746.24","3,492.48","1,433.02","1,090.09","1,498.47","7,334.64"
2,3,32,"76,653.37","14,822.18","1,280.11","60,551.09","1,816.53","1,816.53","3,633.07","1,473.61","1,222.86","1,096.83","11,291.18"
3,4,33,"78,979.24","15,315.59","1,318.95","62,344.70","1,870.34","1,870.34","3,740.68","1,570.49",969.70,835.57,"13,930.35"
4,5,34,"81,299.43","15,802.19","1,357.70","64,139.54","1,924.19","1,924.19","3,848.37","1,519.12","1,267.65","1,055.32","17,755.99"
5,6,35,"82,996.71","16,096.69","1,386.05","65,513.98","1,965.42","1,965.42","3,930.84","1,746.52","1,227.95","1,316.65","23,314.24"
6,7,36,"84,719.10","16,393.35","1,414.81","66,910.95","2,007.33","2,007.33","4,014.66","1,622.37","1,296.71","1,040.20","27,007.33"
7,8,37,"88,002.56","17,152.82","1,469.64","69,380.09","2,081.40","2,081.40","4,162.81","1,677.56","1,291.77","1,220.92","31,348.18"
8,9,38,"92,712.70","18,334.67","1,548.30","72,829.73","2,184.89","2,184.89","4,369.78","1,812.28","1,250.55","1,669.05","38,315.65"
9,10,39,"94,825.77","18,731.61","1,583.59","74,510.57","2,235.32","2,235.32","4,470.63","1,873.99","1,152.40","1,109.03","39,913.31"



🔹 Cumulative Summary:


,Year,Age,Gross Salary,Income Tax,ACC Levy,Net Salary,Employee Contribution,Employer Contribution,Total Contribution,Harboursafe Return,Horizon Return,SkyHigh Return,Total Fund Value
0,Total,N/A,"4,704,736.43","998,130.20","78,569.07","3,628,037.13","108,841.14","108,841.14","217,682.24","89,624.30","69,581.50","70,719.70","536,477.18"



🔹 Every 5-Year Summary:


,Year,Age,Gross Salary,Income Tax,ACC Levy,Net Salary,Employee Contribution,Employer Contribution,Total Contribution,Harboursafe Return,Horizon Return,SkyHigh Return,Total Fund Value
4,5,34,"81,299.43","15,802.19","1,357.70","64,139.54","1,924.19","1,924.19","3,848.37","1,519.12","1,267.65","1,055.32","17,755.99"
9,10,39,"94,825.77","18,731.61","1,583.59","74,510.57","2,235.32","2,235.32","4,470.63","1,873.99","1,152.40","1,109.03","39,913.31"
14,15,44,"114,254.35","23,399.97","1,908.05","88,946.33","2,668.39","2,668.39","5,336.78","2,125.19","1,653.12","1,875.19","81,194.12"
19,20,49,"139,840.26","29,963.66","2,335.33","107,541.26","3,226.24","3,226.24","6,452.48","2,689.01","2,000.79","2,171.01","128,867.59"
24,25,54,"161,797.09","35,082.75","2,702.01","124,012.32","3,720.37","3,720.37","7,440.74","3,131.97","2,438.50","2,463.39","232,717.85"
29,30,59,"191,069.60","42,336.56","3,190.86","145,542.17","4,366.27","4,366.27","8,732.53","3,617.94","2,468.57","3,509.52","376,411.59"
34,35,64,"227,758.66","51,721.65","3,803.57","172,233.44","5,167.00","5,167.00","10,334.01","4,126.73","3,196.88","3,941.30","536,477.18"


In [10]:
import numpy as np
import pandas as pd

# Parameters
initial_salary = 70000
hike_rate = 0.035
self_contribution_rate = 0.03
years = 35
start_age = 30
acc_levy = 0.0167
inflation_rate = 0.025
num_simulations = 5000

# NZ tax brackets
nz_tax_brackets = [
    (0, 15600), (15601, 53500), (53501, 78100), (78101, 180000), (180001, float('inf'))
]
tax_rates = [0.105, 0.175, 0.30, 0.33, 0.39]

# Fund details
funds = {
    "Harboursafe": {"growth_rate": 0.0375, "volatility": 0.05, "weight": 0.4},
    "Horizon": {"growth_rate": 0.065, "volatility": 0.105, "weight": 0.3},
    "SkyHigh": {"growth_rate": 0.1025, "volatility": 0.2075, "weight": 0.3},
}

# Tax function
def calculate_tax(salary, base_slabs, year, inflation_rate=0.025):
    adjusted_slabs = [
        (bracket[0] * ((1 + inflation_rate) ** (year - 1)),
         bracket[1] * ((1 + inflation_rate) ** (year - 1)), rate)
        for (bracket, rate) in zip(base_slabs, tax_rates)
    ]
    tax = 0
    for lower, upper, rate in adjusted_slabs:
        if salary > lower:
            taxable_income = min(salary, upper) - lower
            tax += taxable_income * rate
        else:
            break
    return tax

# Use average Monte Carlo returns for main run
np.random.seed(42)
fund_returns_simulated = {name: [] for name in funds}

for name, fund in funds.items():
    returns = [np.random.normal(fund["growth_rate"], fund["volatility"], num_simulations) for _ in range(years)]
    fund_returns_simulated[name] = np.mean(returns, axis=1)

# Main simulation (single run for reporting)
salary = initial_salary
corpus = 0
records = []

for year in range(1, years + 1):
    age = start_age + year - 1
    if year > 1:
        salary *= (1 + np.random.normal(loc=hike_rate, scale=0.01))

    tax = calculate_tax(salary, nz_tax_brackets, year, inflation_rate)
    acc = acc_levy * salary
    net_salary = salary - tax - acc

    emp_contribution = self_contribution_rate * net_salary
    employer_contribution = 0.03 * net_salary
    total_contribution = emp_contribution + employer_contribution

    harboursafe_return = total_contribution * (1 + fund_returns_simulated["Harboursafe"][year - 1]) * funds["Harboursafe"]["weight"]
    horizon_return = total_contribution * (1 + fund_returns_simulated["Horizon"][year - 1]) * funds["Horizon"]["weight"]
    skyhigh_return = total_contribution * (1 + fund_returns_simulated["SkyHigh"][year - 1]) * funds["SkyHigh"]["weight"]

    total_growth = (
        corpus * (
            fund_returns_simulated["Harboursafe"][year - 1] * funds["Harboursafe"]["weight"] +
            fund_returns_simulated["Horizon"][year - 1] * funds["Horizon"]["weight"] +
            fund_returns_simulated["SkyHigh"][year - 1] * funds["SkyHigh"]["weight"]
        )
    )

    corpus = corpus + total_growth + total_contribution

    records.append({
        "Year": year,
        "Age": age,
        "Gross Salary": round(salary, 2),
        "Income Tax": round(tax, 2),
        "ACC Levy": round(acc, 2),
        "Net Salary": round(net_salary, 2),
        "Employee Contribution": round(emp_contribution, 2),
        "Employer Contribution": round(employer_contribution, 2),
        "Total Contribution": round(total_contribution, 2),
        "Harboursafe Return": round(harboursafe_return, 2),
        "Horizon Return": round(horizon_return, 2),
        "SkyHigh Return": round(skyhigh_return, 2),
        "Total Fund Value": round(corpus, 2)
    })

df_detailed = pd.DataFrame(records)

# Monte Carlo percentiles for final year
all_simulations = []
for sim in range(num_simulations):
    salary = initial_salary
    corpus = 0
    for year in range(1, years + 1):
        if year > 1:
            salary *= (1 + np.random.normal(loc=hike_rate, scale=0.01))
        tax = calculate_tax(salary, nz_tax_brackets, year, inflation_rate)
        acc = acc_levy * salary
        net_salary = salary - tax - acc
        emp_contribution = self_contribution_rate * net_salary
        employer_contribution = 0.03 * net_salary
        total_contribution = emp_contribution + employer_contribution
        weighted_return = sum([
            fund["weight"] * np.random.normal(fund["growth_rate"], fund["volatility"])
            for fund in funds.values()
        ])
        corpus = corpus * (1 + weighted_return) + total_contribution
    all_simulations.append(corpus)

final_corpus_percentiles = np.percentile(all_simulations, [5, 25, 50, 75, 95])
cumulative_summary_df = pd.DataFrame([{
    "Year": "Total",
    "Age": "N/A",
    "5th Percentile": round(final_corpus_percentiles[0], 2),
    "25th Percentile": round(final_corpus_percentiles[1], 2),
    "50th Percentile (Median)": round(final_corpus_percentiles[2], 2),
    "75th Percentile": round(final_corpus_percentiles[3], 2),
    "95th Percentile": round(final_corpus_percentiles[4], 2),
}])

# Every 5-Year Summary
summary_every_5_years = df_detailed[df_detailed["Year"] % 5 == 0]

# Show dataframes
from IPython.display import display
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

print("🔹 Detailed Yearly Summary:")
display(df_detailed)

print("\n🔹 Monte Carlo Cumulative Summary:")
display(cumulative_summary_df)

print("\n🔹 Every 5-Year Summary:")
display(summary_every_5_years)


🔹 Detailed Yearly Summary:


,Year,Age,Gross Salary,Income Tax,ACC Levy,Net Salary,Employee Contribution,Employer Contribution,Total Contribution,Harboursafe Return,Horizon Return,SkyHigh Return,Total Fund Value
0,1,30,"70,000.00","13,220.03","1,169.00","55,610.97","1,668.33","1,668.33","3,336.66","1,385.09","1,067.93","1,105.79","3,336.66"
1,2,31,"72,539.19","13,787.28","1,211.40","57,540.50","1,726.22","1,726.22","3,452.43","1,432.08","1,101.09","1,143.10","7,005.41"
2,3,32,"74,733.97","14,246.35","1,248.06","59,239.56","1,777.19","1,777.19","3,554.37","1,475.82","1,133.17","1,180.37","11,022.93"
3,4,33,"77,383.06","14,836.74","1,292.30","61,254.03","1,837.62","1,837.62","3,675.24","1,526.44","1,175.82","1,216.84","15,429.58"
4,5,34,"79,451.07","15,247.69","1,326.83","62,876.56","1,886.30","1,886.30","3,772.59","1,564.24","1,204.95","1,248.26","20,203.60"
5,6,35,"82,527.39","15,955.89","1,378.21","65,193.29","1,955.80","1,955.80","3,911.60","1,622.80","1,247.71","1,301.06","25,457.93"
6,7,36,"83,854.76","16,134.04","1,400.37","66,320.34","1,989.61","1,989.61","3,979.22","1,650.51","1,270.37","1,316.69","31,089.95"
7,8,37,"86,614.70","16,736.47","1,446.47","68,431.77","2,052.95","2,052.95","4,105.91","1,703.60","1,311.49","1,360.28","37,236.25"
8,9,38,"90,748.06","17,745.27","1,515.49","71,487.29","2,144.62","2,144.62","4,289.24","1,781.58","1,370.75","1,418.23","43,967.65"
9,10,39,"94,961.75","18,772.40","1,585.86","74,603.49","2,238.10","2,238.10","4,476.21","1,857.19","1,432.45","1,475.43","51,281.18"



🔹 Monte Carlo Cumulative Summary:


,Year,Age,5th Percentile,25th Percentile,50th Percentile (Median),75th Percentile,95th Percentile
0,Total,N/A,"396,063.75","508,210.50","606,295.12","721,212.26","943,584.63"



🔹 Every 5-Year Summary:


,Year,Age,Gross Salary,Income Tax,ACC Levy,Net Salary,Employee Contribution,Employer Contribution,Total Contribution,Harboursafe Return,Horizon Return,SkyHigh Return,Total Fund Value
4,5,34,"79,451.07","15,247.69","1,326.83","62,876.56","1,886.30","1,886.30","3,772.59","1,564.24","1,204.95","1,248.26","20,203.60"
9,10,39,"94,961.75","18,772.40","1,585.86","74,603.49","2,238.10","2,238.10","4,476.21","1,857.19","1,432.45","1,475.43","51,281.18"
14,15,44,"108,419.40","21,532.92","1,810.60","85,075.88","2,552.28","2,552.28","5,104.55","2,117.93","1,628.52","1,692.85","97,660.15"
19,20,49,"129,903.67","26,684.59","2,169.39","101,049.69","3,031.49","3,031.49","6,062.98","2,514.11","1,938.05","1,994.75","165,806.55"
24,25,54,"151,108.00","31,555.36","2,523.50","117,029.14","3,510.87","3,510.87","7,021.75","2,911.17","2,242.81","2,313.78","264,282.35"
29,30,59,"185,225.79","40,408.10","3,093.27","141,724.41","4,251.73","4,251.73","8,503.46","3,531.56","2,724.17","2,813.15","406,414.93"
34,35,64,"213,200.92","46,917.59","3,560.46","162,722.87","4,881.69","4,881.69","9,763.37","4,053.07","3,121.00","3,221.49","608,399.23"
